In [1]:
import psycopg2
from psycopg2 import sql
import numpy as np
import pandas as pd
from psycopg2.extensions import adapt, register_adapter, AsIs
from sqlalchemy import create_engine, text
import getpass
from scipy.stats import ttest_ind, zscore
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from itertools import product

In [ ]:
mypasswd = getpass.getpass("Enter your database password: ")
username = 'jsmm8'
host = 'pgsql'
database = 'f24t03'

conn_string = f"postgresql+psycopg2://{username}:{mypasswd}@{host}/{database}"
engine = create_engine(conn_string)

Enter your database password: ········


In [3]:
select_query = "SELECT * FROM demographics_master;" 

with engine.connect() as connection:
    result = connection.execute(text(select_query))
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
print("Query executed successfully.")
df

Query executed successfully.


,geo_id,year,total_population,shannon_index,simpsons_index,richness,simpson_dominance,berger_parker_dominance,simpson_evenness,block_group,...,est_35to40k,est_40to45k,est_45to50k,est_50to60k,est_60to75k,est_75to100k,est_100to125k,est_125to150k,est_150to200k,est_more_than_200k
0,1500000US290019501001,2020,1284,0.058,0.031,0.500,0.969,0.984,0.041,Block Group 1,...,0.14,0.19,0.11,0.70,0.24,0.39,0.31,0.14,0.05,0.03
1,1500000US290019501002,2020,1403,0.093,0.058,0.375,0.942,0.970,0.088,Block Group 2,...,0.04,0.16,0.09,0.25,0.46,0.40,0.26,0.12,0.24,0.02
2,1500000US290019501003,2020,1007,0.284,0.266,0.625,0.734,0.843,0.333,Block Group 3,...,0.10,0.09,0.31,0.43,0.38,0.38,0.09,0.16,0.15,0.09
3,1500000US290019502001,2020,891,0.201,0.140,0.500,0.860,0.926,0.187,Block Group 1,...,0.20,0.07,0.12,0.13,0.23,0.55,0.24,0.00,0.00,0.03
4,1500000US290019502002,2020,991,0.215,0.166,0.500,0.834,0.910,0.221,Block Group 2,...,0.14,0.18,0.04,0.41,0.35,0.38,0.25,0.17,0.33,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15088,1500000US295101277001,2022,697,0.085,0.061,0.250,0.939,0.968,0.122,Block Group 1,...,0.04,0.22,0.00,0.38,0.05,0.05,0.14,0.04,0.00,0.00
15089,1500000US295101277002,2022,824,0.184,0.163,0.250,0.837,0.910,0.327,Block Group 2,...,0.00,0.00,0.00,0.06,0.18,0.00,0.00,0.00,0.00,0.00
15090,1500000US295101277003,2022,645,0.049,0.031,0.250,0.969,0.984,0.061,Block Group 3,...,0.00,0.00,0.12,0.79,0.21,0.09,0.00,0.00,0.00,0.00
15091,1500000US295101278001,2022,1824,0.467,0.380,0.625,0.620,0.771,0.475,Block Group 1,...,0.02,0.19,0.16,0.05,0.08,0.08,0.00,0.00,0.00,0.00


In [4]:
df.columns.tolist()

['geo_id',
 'year',
 'total_population',
 'shannon_index',
 'simpsons_index',
 'richness',
 'simpson_dominance',
 'berger_parker_dominance',
 'simpson_evenness',
 'block_group',
 'census_tract',
 'county',
 'state',
 'group_classification',
 'hispanic_or_latino',
 'asian',
 'black',
 'native',
 'other',
 'pac_isl',
 'two_or_more_races',
 'white',
 'gini_education',
 'weighted_education_score',
 'higher_education_proportion',
 'education_score_category',
 'no_schooling',
 'prek_8th',
 'some_high_school_no_diploma',
 'high_school_graduate',
 'some_college_no_degree',
 'associates_degree',
 'bachelors_degree',
 'advanced_degree',
 'median_income',
 'gini_coefficient',
 'est_less_than_10k',
 'est_10to15k',
 'est_15to20k',
 'est_20to25k',
 'est_25to30k',
 'est_30to35k',
 'est_35to40k',
 'est_40to45k',
 'est_45to50k',
 'est_50to60k',
 'est_60to75k',
 'est_75to100k',
 'est_100to125k',
 'est_125to150k',
 'est_150to200k',
 'est_more_than_200k']

# Year 2020 Analysis

## Analysis of Income Disparities by Diversity Level and Dominant Race

To perform hypothesis testing, we analyze whether there are significant differences in median income between communities with "low diversity" and "high diversity" for various dominant racial groups in the dataset.

#### Hypotheses
##### Null Hypothesis (H0): 
There is no significant difference in the median income of communities with low diversity and high diversity for a given dominant racial group.

##### Alternative Hypothesis (𝐻𝑎): 
There is a significant difference in the median income of communities with low diversity and high diversity for a given dominant racial group.

#### Testing Method

##### T-test: 
The code uses an independent samples t-test to compare the median incomes between low diversity and high diversity groups for each dominant racial group.

𝐻0 : 𝜇𝑙𝑜𝑤 = 𝜇ℎ𝑖𝑔ℎ

𝐻𝑎 : 𝜇𝑙𝑜𝑤 ≠𝜇ℎ𝑖𝑔ℎ

In [5]:
df_2020 = df[(df['year'] == 2020) & (df['gini_coefficient'] != -1)].dropna(subset=['median_income', 'group_classification'])

df_2020['dominant_race'] = df_2020['group_classification'].str.extract(r'^(.*?) Dominant')
df_2020['diversity_level'] = df_2020['group_classification'].str.extract(r'with (low|high) diversity')

income_race_results = []

population_threshold = 30

for race in df_2020['dominant_race'].dropna().unique():
    low_diversity = df_2020[
        (df_2020['dominant_race'] == race) & 
        (df_2020['diversity_level'] == 'low')
    ]['median_income']
    
    high_diversity = df_2020[
        (df_2020['dominant_race'] == race) & 
        (df_2020['diversity_level'] == 'high')
    ]['median_income']
    
    if len(low_diversity) >= population_threshold and len(high_diversity) >= population_threshold:
        t_stat, p_val = ttest_ind(low_diversity, high_diversity, nan_policy='omit')

        combined_data = pd.concat([low_diversity, high_diversity])
        z_scores = zscore(combined_data)

        low_diversity_median = low_diversity.median()
        high_diversity_median = high_diversity.median()

        income_race_results.append({
            "Race": race,
            "Low Diversity": low_diversity_median,
            "High Diversity": high_diversity_median,
            "T-Statistic": t_stat,
            "P-Value": p_val,
            "Mean Z-Score": z_scores.mean() if len(z_scores) > 0 else None
        })

income_race_results_df = pd.DataFrame(income_race_results)
income_race_results_df

,Race,Low Diversity,High Diversity,T-Statistic,P-Value,Mean Z-Score
0,white,55966.0,60653.5,-3.860957,0.000115,-9.251428e-17
1,black,35583.0,39366.5,-2.544801,0.011247,-1.181776e-16


The analysis examined the relationship between diversity levels within communities and median income, stratified by the dominant racial group. For white-dominated communities, the median income for low-diversity areas was 55,966, compared to 60,653.5 in high-diversity areas. This difference was statistically significant, with a t-statistic of -3.86 and a p-value of 0.000115, indicating a strong association between higher diversity and increased median income. The mean z-score, close to zero, suggests that the income data points are centered relative to the distribution.

Similarly, black-dominated communities showed a lower median income in low-diversity areas (35,583) compared to high-diversity areas (39,366.5). This income difference was also statistically significant, with a t-statistic of -2.54 and a p-value of 0.011247. These findings underscore the potential positive impact of diversity on income levels, as communities with higher diversity consistently exhibited higher median incomes for both racial groups.

These results suggest that greater diversity correlates with economic advantages in terms of median income across different racial compositions. While the analysis is limited to 2020 and specific population thresholds, the findings align with broader discussions about the socioeconomic benefits of diversity.

## Analysis of Weighted Education Scores by Race and Diversity Level

In [6]:
def classify_p_value(p_val):
    if p_val <= 0.01:
        return "Highly Significant"
    elif 0.01 < p_val <= 0.05:
        return "Significant"
    elif 0.05 < p_val <= 0.1:
        return "Marginally Significant"
    else:
        return "Not Significant"
df_2020 = df[df['year'] == 2020].copy()

diversity_threshold = df_2020['shannon_index'].median()
df_2020.loc[:, 'diversity_level'] = df_2020['shannon_index'].apply(
    lambda x: 'high' if x > diversity_threshold else 'low'
)

df_2020.loc[:, 'dominant_race'] = df_2020['group_classification'].str.extract(r'^(.*?) Dominant')

group_comparisons = [
    ("White", "high", "White", "low"),
    ("Black", "high", "Black", "low"),
    ("White", "high", "Black", "high"),
    ("White", "high", "Black", "low"),
    ("White", "low", "Black", "high"),
    ("White", "low", "Black", "low"),
]

results = []

for race1, div1, race2, div2 in group_comparisons:
    group1 = df_2020[
        (df_2020['group_classification'].str.contains(race1, case=False, na=False)) &
        (df_2020['diversity_level'] == div1)
    ]['weighted_education_score']
    group2 = df_2020[
        (df_2020['group_classification'].str.contains(race2, case=False, na=False)) &
        (df_2020['diversity_level'] == div2)
    ]['weighted_education_score']
    
    if len(group1) > 0 and len(group2) > 0:
        t_stat, p_val = ttest_ind(group1, group2, nan_policy='omit')
        results.append({
            "Diversity Level 1": f"{race1.capitalize()} {div1.capitalize()}",
            "Diversity Level 2": f"{race2.capitalize()} {div2.capitalize()}",
            "T-Statistic": t_stat,
            "P-Value": p_val,
            "Significance": classify_p_value(p_val),
        })

results_df = pd.DataFrame(results)
results_df

,Diversity Level 1,Diversity Level 2,T-Statistic,P-Value,Significance
0,White High,White Low,13.795087,2.153655e-42,Highly Significant
1,Black High,Black Low,5.377524,1.182624e-07,Highly Significant
2,White High,Black High,11.028597,1.366631e-27,Highly Significant
3,White High,Black Low,13.214767,2.143317e-38,Highly Significant
4,White Low,Black High,5.072615,4.197876e-07,Highly Significant
5,White Low,Black Low,9.170811,9.533081e-20,Highly Significant


The study compared weighted education scores across different racial and diversity-level groupings using t-tests to evaluate the statistical significance of differences. Across all comparisons, the results were classified as "Highly Significant," with p-values far below the threshold of 0.01, indicating strong evidence of differences between groups.

#### Intra-Racial Comparisons by Diversity Level:
Among white-dominated communities, high-diversity areas exhibited significantly higher weighted education scores than low-diversity areas, with a t-statistic of 13.80 and a p-value of 2.15×10−42. Similarly, for black-dominated communities, high-diversity areas also had significantly higher education scores than low-diversity areas, with a t-statistic of 5.38 and a p-value of 1.18×10−7. These findings suggest that diversity correlates positively with educational outcomes within racially homogeneous groups.

#### Inter-Racial Comparisons:
When comparing white high-diversity to black high-diversity areas, white communities exhibited significantly higher weighted education scores, with a t-statistic of 11.03 and a p-value of 1.37×10−27. Comparisons between white high-diversity and black low-diversity areas revealed even more pronounced differences (t-statistic: 13.21, p-value: 2.14×10−38), emphasizing disparities not only in diversity but also in racial groupings.

#### Cross-Diversity and Cross-Race Comparisons:
Between white low-diversity and black high-diversity areas, white communities maintained higher education scores (t-statistic: 5.07, p-value: 4.20×10−7). Lastly, white low-diversity areas also had significantly higher education scores compared to black low-diversity areas, with a t-statistic of 9.17 and a p-value of 9.53×10−20.

#### Implications
These results reveal stark disparities in educational outcomes tied to both racial and diversity-level classifications. High-diversity communities, regardless of racial composition, tend to perform better in terms of weighted education scores, suggesting that diversity may serve as a positive factor in educational attainment. However, the consistently higher scores in white communities compared to black communities across all diversity levels underscore systemic racial inequities in education. 

## Analysis of Education Score Categories by Diversity Levels

In [7]:
unique_education_scores = df_2020['education_score_category'].unique()
unique_diversity_levels = df_2020['diversity_level'].unique()

education_diversity_counts = df_2020.groupby(['education_score_category', 'diversity_level']).size()

unique_education_scores, unique_diversity_levels, education_diversity_counts

(array(['Medium', 'Low', 'High', 'Very High', 'Very Low', 'nan'],
       dtype=object),
 array(['low', 'high'], dtype=object),
 education_score_category  diversity_level
 High                      high                572
                           low                 320
 Low                       high                695
                           low                 882
 Medium                    high               1075
                           low                1219
 Very High                 high                148
                           low                  66
 Very Low                  high                 20
                           low                  16
 nan                       low                  18
 dtype: int64)

The distribution of education score categories across high- and low-diversity levels provides insights into how diversity correlates with educational outcomes:

#### Unique Categories:
The dataset includes five primary education score categories: "Very Low," "Low," "Medium," "High," and "Very High," with an additional "nan" category representing missing or undefined values.
Diversity levels are divided into "high" and "low," based on the Shannon diversity index threshold.

#### Category Distribution:
The Medium education score category is the most prevalent, with 1,075 cases in high-diversity areas and 1,219 cases in low-diversity areas, highlighting its dominance regardless of diversity.

The High education score category appears more frequently in high-diversity areas (572) compared to low-diversity areas (320), indicating a positive relationship between high diversity and better education outcomes.
Conversely, the Low education score category is more common in low-diversity areas (882) than in high-diversity areas (695), suggesting a negative association between low diversity and educational attainment.

#### Extremes in Education Scores:
The Very High education score category is more prevalent in high-diversity areas (148 cases) compared to low-diversity areas (66), reinforcing the trend that high diversity supports better educational outcomes.

The Very Low education score category is rare, but high-diversity areas still have a slight edge (20 cases) over low-diversity areas (16), reflecting a smaller but notable effect of diversity on minimizing the lowest scores.

#### Missing Data:
A small number of "nan" entries (18 cases) appear in low-diversity areas, which might indicate either data entry issues or areas with undefined education score categories.

#### Implications
The results suggest a strong association between diversity levels and education outcomes. High-diversity areas tend to exhibit higher proportions of positive educational outcomes, as evidenced by higher frequencies of "High" and "Very High" education score categories. Meanwhile, low-diversity areas show a greater concentration of "Low" education scores, underscoring the potential impact of diversity on educational success. Addressing the structural factors that create these disparities, such as resource allocation and community inclusiveness, could help bridge the gap in educational equity.

## Analysis of Median Income by Education Level and Inequality

In [8]:
df_2020 = df[(df['year'] == 2020) & (df['gini_education'] != -1)].dropna(subset=['median_income', 'education_score_category'])

income_education_results = []

population_threshold = 30

for education_level in df_2020['education_score_category'].dropna().unique():
    low_education = df_2020[
        (df_2020['education_score_category'] == education_level) & 
        (df_2020['gini_education'] < df_2020['gini_education'].median())
    ]['median_income']
    
    high_education = df_2020[
        (df_2020['education_score_category'] == education_level) & 
        (df_2020['gini_education'] >= df_2020['gini_education'].median())
    ]['median_income']

    if len(low_education) >= population_threshold and len(high_education) >= population_threshold:
        t_stat, p_val = ttest_ind(low_education, high_education, nan_policy='omit')

        combined_data = pd.concat([low_education, high_education])
        z_scores = zscore(combined_data)

        low_education_median = low_education.median()
        high_education_median = high_education.median()

        income_education_results.append({
            "Education Level": education_level,
            "Low Inequality": low_education_median,
            "High Inequality": high_education_median,
            "T-Statistic": t_stat,
            "P-Value": p_val,
            "Mean Z-Score": z_scores.mean() if len(z_scores) > 0 else None
        })

income_education_results_df = pd.DataFrame(income_education_results)
income_education_results_df

,Education Level,Low Inequality,High Inequality,T-Statistic,P-Value,Mean Z-Score
0,Medium,59069.0,55192.0,3.888012,0.000104,1.177011e-16
1,Low,41042.0,40000.0,1.082166,0.279345,-1.531925e-16
2,High,79710.0,87852.0,-3.983213,0.000074,1.752460e-16


The analysis investigates the relationship between education levels, inequality (as measured by the Gini coefficient), and median income. The results are stratified by low and high inequality within each education level category and assessed for statistical significance.

#### Medium Education Level:
Communities with medium education scores and low inequality exhibited a higher median income (59,069) compared to those with high inequality (55,192). This difference was statistically significant, with a t-statistic of 3.89 and a p-value of 0.000104, indicating that income disparities are strongly associated with inequality even within this education category.
The mean z-score near zero suggests the income data points are centered relative to the distribution, ensuring validity of the results.

#### Low Education Level:
For communities with low education scores, the median income was slightly higher in low-inequality areas (41,042) compared to high-inequality areas (40,000). However, this difference was not statistically significant, with a t-statistic of 1.08 and a p-value of 0.279. This indicates that within the low education category, inequality may have a less pronounced impact on income.

#### High Education Level:
In communities with high education scores, the trend reversed. High-inequality areas showed a higher median income (87,852) than low-inequality areas (79,710). This difference was statistically significant, with a t-statistic of -3.98 and a p-value of 0.000074, suggesting that within high-education communities, inequality might relate to an increase in income disparities favoring higher earners.

#### Implications
The findings highlight a complex relationship between education levels, inequality, and income. While low inequality generally corresponds to higher median income in communities with medium education levels, this trend does not hold consistently across all education levels. For high-education communities, high inequality is associated with higher median income, suggesting a possible stratification effect where a subset of earners significantly boosts the median income.

## Analysis of Pairwise Comparisons Between Education and Gini Levels

In [9]:
group_comparisons = [
    ("Low", "Low", "Low", "High"),
    ("Medium", "Low", "Medium", "High"),
    ("High", "Low", "High", "High"),
    ("Low", "High", "Medium", "High"),
    ("Medium", "High", "High", "High"),
]

pairwise_results = []

for edu1, gini1, edu2, gini2 in group_comparisons:
    group1 = df_2020[
        (df_2020['education_score_category'] == edu1) & 
        ((df_2020['gini_education'] < df_2020['gini_education'].median()) if gini1 == "Low" else (df_2020['gini_education'] >= df_2020['gini_education'].median()))
    ]['median_income']
    
    group2 = df_2020[
        (df_2020['education_score_category'] == edu2) & 
        ((df_2020['gini_education'] < df_2020['gini_education'].median()) if gini2 == "Low" else (df_2020['gini_education'] >= df_2020['gini_education'].median()))
    ]['median_income']

    if len(group1) > 0 and len(group2) > 0:
        t_stat, p_val = ttest_ind(group1, group2, nan_policy='omit')
        pairwise_results.append({
            "Education Level 1": f"{edu1} Education - {gini1} Inequality",
            "Education Level 2": f"{edu2} Education - {gini2} Inequality",
            "T-Statistic": t_stat,
            "P-Value": p_val,
        })

pairwise_results_df = pd.DataFrame(pairwise_results)
pairwise_results_df

,Education Level 1,Education Level 2,T-Statistic,P-Value
0,Low Education - Low Inequality,Low Education - High Inequality,1.082166,2.793445e-01
1,Medium Education - Low Inequality,Medium Education - High Inequality,3.888012,1.039418e-04
2,High Education - Low Inequality,High Education - High Inequality,-3.983213,7.354429e-05
3,Low Education - High Inequality,Medium Education - High Inequality,-18.690336,7.570446e-72
4,Medium Education - High Inequality,High Education - High Inequality,-19.552892,4.191588e-75


This analysis evaluates the differences in median income between various education levels and Gini inequality categories. Pairwise comparisons assess whether statistically significant differences exist between groups, providing insights into the interplay between inequality and educational attainment.

### Intra-Education Comparisons:
##### Low Education, Low vs. High Inequality: 
The comparison between low-inequality and high-inequality groups within the low education category yielded a t-statistic of 1.08 and a p-value of 0.279. This result is not statistically significant, indicating that inequality may not strongly affect median income in low-education groups.

##### Medium Education, Low vs. High Inequality: 
The medium education group showed a significant difference, with a t-statistic of 3.89 and a p-value of 1.04×10 −4. This suggests that inequality has a notable impact on median income for this category.

#####  High Education, Low vs. High Inequality:
Within the high education group, high-inequality areas exhibited significantly higher median incomes than low-inequality areas (t-statistic: -3.98, p-value: 7.35×10−5).

### Inter-Education Comparisons:
##### Low Education (High Inequality) vs. Medium Education (High Inequality): 
This comparison yielded a t-statistic of -18.69 and a p-value of 7.57×10 −72, indicating a highly significant difference in median incomes. Median income increases substantially as education level transitions from low to medium, even within high-inequality contexts.

##### Medium Education (High Inequality) vs. High Education (High Inequality): 
Similarly, this comparison produced a t-statistic of -19.55 and a p-value of 4.19×10−75, reinforcing that educational attainment strongly correlates with higher median income, regardless of inequality.

### Implications
The results reveal several important trends:

##### Inequality's Role: 
While inequality has a limited impact on median income within the low education category, it significantly affects incomes in medium and high education categories. This suggests that the benefits of higher education may be amplified in more equitable contexts.

##### Education as a Determinant:
Median income consistently increases with higher education levels across both low and high inequality scenarios. The stark differences in inter-education comparisons highlight education's critical role in promoting economic mobility.

##### Targeted Policies:
These findings support the need for policies addressing both educational attainment and income inequality, particularly targeting low-education, high-inequality groups to improve economic outcomes. Structural reforms in education access and quality could mitigate disparities and promote broader equity.

## Comprehensive Analysis of Education Levels and Gini Inequality Impact on Median Income

In [10]:
def classify_p_value(p_val):
    if p_val <= 0.01:
        return "Highly Significant"
    elif 0.01 < p_val <= 0.05:
        return "Significant"
    elif 0.05 < p_val <= 0.1:
        return "Marginally Significant"
    else:
        return "Not Significant"

df_2020 = df[(df['year'] == 2020) & (df['gini_education'] != -1)].dropna(subset=['median_income', 'education_score_category'])

education_levels = df_2020['education_score_category'].unique()
gini_categories = ["low", "high"]

group_comparisons = list(product(education_levels, gini_categories, education_levels, gini_categories))

education_results = []

gini_median = df_2020['gini_education'].median()

for edu1, gini1, edu2, gini2 in group_comparisons:
    # Skip self-comparisons
    if edu1 == edu2 and gini1 == gini2:
        continue

    group1 = df_2020[
        (df_2020['education_score_category'] == edu1) &
        ((df_2020['gini_education'] < gini_median) if gini1 == "low" else (df_2020['gini_education'] >= gini_median))
    ]['median_income']
    
    group2 = df_2020[
        (df_2020['education_score_category'] == edu2) &
        ((df_2020['gini_education'] < gini_median) if gini2 == "low" else (df_2020['gini_education'] >= gini_median))
    ]['median_income']
    
    if len(group1) > 0 and len(group2) > 0:
        t_stat, p_val = ttest_ind(group1, group2, nan_policy='omit')
        education_results.append({
            "Education Level 1": f"{edu1} Education Level ({'Low' if gini1 == 'low' else 'High'})",
            "Education Level 2": f"{edu2} Education Level ({'Low' if gini2 == 'low' else 'High'})",
            "T-Statistic": t_stat,
            "P-Value": p_val,
            "Significance": classify_p_value(p_val),
        })

cleaned_results = []
seen_combinations = set()

for result in education_results:
    combo = tuple(sorted([result["Education Level 1"], result["Education Level 2"]]))
    if combo not in seen_combinations:
        seen_combinations.add(combo)
        cleaned_results.append(result)

unique_education_results_df = pd.DataFrame(cleaned_results)
unique_education_results_df

,Education Level 1,Education Level 2,T-Statistic,P-Value,Significance
0,Medium Education Level (Low),Medium Education Level (High),3.888012,1.039418e-04,Highly Significant
1,Medium Education Level (Low),Low Education Level (Low),18.903205,3.306291e-73,Highly Significant
2,Medium Education Level (Low),Low Education Level (High),24.891432,6.053457e-122,Highly Significant
3,Medium Education Level (Low),High Education Level (Low),-16.272434,9.909193e-56,Highly Significant
4,Medium Education Level (Low),High Education Level (High),-19.755772,8.350920e-79,Highly Significant
5,Medium Education Level (Low),Very High Education Level (High),-27.797065,6.061350e-139,Highly Significant
6,Medium Education Level (Low),Very Low Education Level (Low),3.935536,8.712069e-05,Highly Significant
7,Medium Education Level (Low),Very Low Education Level (High),6.712702,2.766363e-11,Highly Significant
8,Medium Education Level (High),Low Education Level (Low),14.487724,1.591083e-44,Highly Significant
9,Medium Education Level (High),Low Education Level (High),18.690336,7.570446e-72,Highly Significant


The pairwise comparisons explore the relationship between education levels, Gini inequality categories, and their impact on median income. Key findings emerge from these highly significant statistical results, revealing disparities and trends based on the interplay of education and inequality.

### Intra-Education Comparisons
##### Medium Education Level (Low vs. High Inequality):
A statistically significant difference was observed between low-inequality ($p = 0.0001$) and high-inequality groups, where low-inequality groups generally exhibited higher median incomes. This underscores the benefits of reduced inequality within the medium education category.

##### Low Education Level (Low vs. High Inequality):
Results were not significant ($p = 0.279$), suggesting minimal income disparity between low- and high-inequality groups in this category. This highlights the weaker influence of inequality in the low education bracket.

##### High Education Level (Low vs. High Inequality):
A significant difference ($p = 0.00007$) shows that high-inequality groups exhibit higher median incomes, indicating that inequality amplifies income disparities within this category.
Inter-Education Comparisons

##### Low vs. Medium Education Levels:
Across both low- and high-inequality categories, medium education groups consistently showed significantly higher incomes than low-education groups ($p < 0.0001$). This emphasizes the economic benefits of advancing from low to medium education levels.

##### Medium vs. High Education Levels:
High education groups outperformed medium education groups in income, particularly in high-inequality scenarios ($p < 0.0001$). However, low-inequality settings yielded less pronounced differences, showcasing the compounding effect of education and inequality on income.

##### Very High Education Levels:
Comparisons involving very high education levels consistently showed significant advantages over lower education levels, reinforcing the strong association between advanced education and economic benefits.
Edge Cases and Notable Trends

##### Very Low Education Level Comparisons:
These groups showed mixed results, with significant differences when compared to medium, high, and very high education levels but often no significant difference in intra-group comparisons (e.g., low vs. high inequality, $p = 0.557$).

##### Amplifying Role of Inequality:
High-inequality scenarios consistently yielded larger income gaps across all education levels. This suggests that inequality exacerbates income stratification and disproportionately benefits those in higher education categories.

#### Implications
These findings highlight the interplay between education and inequality in shaping economic outcomes:

$Education as an Equalizer:$ Advancing education levels significantly improves median income across all scenarios, underscoring the importance of investment in education.

$Impact of Inequality:$
While inequality has minimal impact within low education groups, it becomes increasingly relevant for medium and high education levels, where it amplifies disparities.

## Summary of Income Distribution by Education Score Categories (2020)

In [11]:
education_category_analysis_2020 = df_2020.groupby('education_score_category').agg(
    Median_Income=('median_income', 'median'),
    Std_Income=('median_income', 'std')
).reset_index()

education_category_analysis_2020

,education_score_category,Median_Income,Std_Income
0,High,82297.0,33709.176088
1,Low,40368.0,15371.820949
2,Medium,57578.0,21353.206976
3,Very High,117779.5,80594.171907
4,Very Low,33137.5,15613.491170


This analysis examines the median income and standard deviation for various education score categories in 2020. The findings highlight significant variations in income levels and variability across categories.

### Key Observations
#### Median Income:
##### Very High Education: 
The highest median income of 117,779.5 is observed in the "Very High" education category, reflecting substantial economic benefits associated with top-tier education.

High Education: This category ranks second with a median income of 82,297, emphasizing the financial advantages of achieving a high education level.

Medium Education: The median income drops to 57,578, highlighting a middle-tier economic position for this category.

Low and Very Low Education: The lowest medians are 40,368 and 33,137.5, respectively, underscoring the economic challenges faced by individuals with lower educational attainment.

#### Income Variability (Standard Deviation):

##### Very High Education: 
The largest standard deviation (80,594.17) suggests substantial variability in income, likely influenced by high earners in this category.

##### High and Medium Education:
These categories show moderate variability (33,709.18 and 21,353.21), indicating relatively consistent income distributions compared to the "Very High" category.

##### Low and Very Low Education: 
The smallest standard deviations (15,371.82 and 15,613.49) reflect less income diversity, consistent with limited earning potential in these groups.

#### Implications
##### Economic Returns of Education:
Higher education levels are strongly correlated with higher median incomes, with "Very High" and "High" categories significantly outpacing "Medium," "Low," and "Very Low" groups.

##### Income Inequality:
The substantial standard deviation in the "Very High" education category suggests a broader income range, indicating potential inequality even among highly educated groups.

## ANOVA Analysis of Median Income by Diversity Levels (2020)

In [12]:
df_2020_cleaned = df[(df['year'] == 2020) & (df['higher_education_proportion'] >= 0)].dropna(
    subset=['higher_education_proportion', 'group_classification', 'median_income']
)

group_classifications = df_2020_cleaned['group_classification'].unique()

anova_results_with_significance = []

def classify_p_value(p_val):
    if p_val <= 0.01:
        return "Highly Significant"
    elif 0.01 < p_val <= 0.05:
        return "Significant"
    elif 0.05 < p_val <= 0.1:
        return "Marginally Significant"
    else:
        return "Not Significant"

for group1 in group_classifications:
    for group2 in group_classifications:
        if group1 != group2:
            group1_data = df_2020_cleaned[df_2020_cleaned['group_classification'] == group1]['median_income']
            group2_data = df_2020_cleaned[df_2020_cleaned['group_classification'] == group2]['median_income']

            if len(group1_data) > 30 and len(group2_data) > 30:
           
                t_stat, p_val = ttest_ind(group1_data, group2_data, nan_policy='omit')

                combined_data = pd.concat([group1_data, group2_data])
                z_scores = zscore(combined_data)

                anova_results_with_significance.append({
                    "Diversity Level": group1,
                    "Diversity Level": group2,
                    "T-Statistic": t_stat,
                    "P-Value": p_val,
                    "Mean Z-Score": z_scores.mean() if len(z_scores) > 0 else None,
                    "Significance": classify_p_value(p_val)
                })

anova_results_with_significance_df = pd.DataFrame(anova_results_with_significance)
anova_results_with_significance_df

,Diversity Level,T-Statistic,P-Value,Mean Z-Score,Significance
0,white Dominant with high diversity,-3.770535,1.650849e-04,-2.642037e-17,Highly Significant
1,black Dominant with high diversity,5.117460,3.283904e-07,-9.998043e-17,Highly Significant
2,Mixed Race Dominant,5.000791,6.013835e-07,5.273044e-17,Highly Significant
3,black Dominant with low diversity,13.577628,6.133472e-41,1.286614e-16,Highly Significant
4,white Dominant with low diversity,3.770535,1.650849e-04,-2.642037e-17,Highly Significant
5,black Dominant with high diversity,5.638324,2.086061e-08,6.269495e-17,Highly Significant
6,Mixed Race Dominant,5.914922,4.126120e-09,5.812211e-17,Highly Significant
7,black Dominant with low diversity,14.142488,5.926597e-43,8.147442e-17,Highly Significant
8,white Dominant with low diversity,-5.117460,3.283904e-07,-1.181587e-16,Highly Significant
9,white Dominant with high diversity,-5.638324,2.086061e-08,6.269495e-17,Highly Significant


The ANOVA analysis investigates differences in median income across diverse group classifications based on higher education proportions. Results are categorized by statistical significance to identify key trends and disparities.

#### Highly Significant Comparisons
##### White-Dominant vs. Black-Dominant (Low and High Diversity):

Comparisons between "white dominant with low diversity" and "black dominant with low diversity" yield highly significant differences (e.g., t = −13.58, p < 0.0001). Similar trends are observed for high diversity groups (t = −14.14, p < 0.0001).

These findings highlight notable income disparities between racial groups, regardless of diversity levels.

##### Black-Dominant vs. Mixed-Race Groups:
Both "black dominant with high diversity" and "low diversity" groups show highly significant differences when compared to "Mixed Race Dominant" groups ( t= 5.59, p < 0.0001).

Mixed-race groups consistently show income advantages over black-dominant groups, emphasizing systemic disparities.

##### White-Dominant with High vs. Low Diversity:
Within white-dominant communities, those with high diversity exhibit significantly higher median incomes compared to low-diversity groups (t = −5.64, p < 0.0001).
This reinforces the positive impact of diversity on economic outcomes within this classification.
Significant and Not Significant Comparisons

##### Black-Dominant (High vs. Low Diversity):
A significant difference (t = −2.54, p = 0.011) exists between high and low diversity in black-dominant groups. This indicates that diversity positively influences income but to a lesser extent than observed in white-dominant groups.

##### Mixed Race vs. Other Groups:
Comparisons between "Mixed Race Dominant" and other categories are often less pronounced, with some results classified as "Not Significant" (t = −1.60, p = 0.11). This suggests relative parity in income within mixed-race groups.

#### Implications

##### Racial and Diversity Disparities:
The analysis underscores significant racial income disparities, particularly between white-dominant and black-dominant groups, regardless of diversity level. Structural inequities may contribute to these findings.

##### Role of Diversity:
Diversity correlates positively with income, particularly in white-dominant groups. Strategies to foster inclusivity could help address income gaps in less diverse areas.

##### Policy Recommendations:
Efforts to improve economic equity should focus on supporting black-dominant and low-diversity communities. This could involve investments in education, employment opportunities, and infrastructure.

This comprehensive analysis highlights critical income inequalities tied to diversity and race, emphasizing the need for systemic reforms to achieve economic parity.

# Year 2021 Analysis

## Analysis of Median Income by Dominant Race and Diversity Levels (2021)

In [13]:
df_2021 = df[(df['year'] == 2021) & (df['gini_coefficient'] != -1)].dropna(subset=['median_income', 'group_classification'])

df_2021['dominant_race'] = df_2021['group_classification'].str.extract(r'^(.*?) Dominant')
df_2021['diversity_level'] = df_2021['group_classification'].str.extract(r'with (low|high) diversity')

income_race_results_1 = []

population_threshold = 30

for race in df_2021['dominant_race'].dropna().unique():
    low_diversity = df_2021[
        (df_2021['dominant_race'] == race) & 
        (df_2021['diversity_level'] == 'low')
    ]['median_income']
    
    high_diversity = df_2021[
        (df_2021['dominant_race'] == race) & 
        (df_2021['diversity_level'] == 'high')
    ]['median_income']
    
    if len(low_diversity) >= population_threshold and len(high_diversity) >= population_threshold:
        t_stat, p_val = ttest_ind(low_diversity, high_diversity, nan_policy='omit')

        combined_data = pd.concat([low_diversity, high_diversity])
        z_scores = zscore(combined_data)

        low_diversity_median = low_diversity.median()
        high_diversity_median = high_diversity.median()

        income_race_results_1.append({
            "Dominant_Race": race,
            "Low_Diversity_Median": low_diversity_median,
            "High_Diversity_Median": high_diversity_median,
            "T-Statistic": t_stat,
            "P-Value": p_val,
            "Mean Z-Score": z_scores.mean() if len(z_scores) > 0 else None
        })

income_race_results_1_df = pd.DataFrame(income_race_results)
income_race_results_1_df

,Race,Low Diversity,High Diversity,T-Statistic,P-Value,Mean Z-Score
0,white,55966.0,60653.5,-3.860957,0.000115,-9.251428e-17
1,black,35583.0,39366.5,-2.544801,0.011247,-1.181776e-16


This analysis examines the differences in median income for dominant racial groups across low- and high-diversity settings, focusing on white- and black-dominant communities. Key statistical measures, including t-tests, reveal significant disparities.

### Findings
##### White-Dominant Communities:

Low Diversity: The median income was $55,966.

High Diversity: The median income increased to $60,653.5, reflecting a positive correlation between diversity and income.

Statistical Significance: The t-statistic of -3.86 and a p-value of 0.000115 indicate a highly significant difference, suggesting that higher diversity significantly boosts income levels within white-dominant communities.

##### Black-Dominant Communities:

Low Diversity: The median income was $35,583.

High Diversity: This rose to $39,366.5, also indicating a positive impact of diversity on income.

Statistical Significance: A t-statistic of -2.54 and a p-value of 0.011247 reveal a significant difference, though less pronounced than in white-dominant communities.

#### Mean Z-Scores:

The near-zero mean z-scores (-9.25e-17 for white and -1.18e-16 for black communities) confirm that the income distributions are centered and appropriately normalized for comparison.

### Implications

#### Impact of Diversity:
Both white- and black-dominant communities benefit from increased diversity in terms of higher median incomes. However, the effect is more pronounced in white-dominant areas, highlighting potential structural advantages in these communities.

##### Racial Disparities:
Median incomes in black-dominant communities remain significantly lower than in white-dominant ones, even within high-diversity settings. This underscores persistent racial income inequalities that merit targeted interventions.

## Analysis of Weighted Education Scores by Race and Diversity Levels (2021)

In [14]:
def classify_p_value(p_val):
    if p_val <= 0.01:
        return "Highly Significant"
    elif 0.01 < p_val <= 0.05:
        return "Significant"
    elif 0.05 < p_val <= 0.1:
        return "Marginally Significant"
    else:
        return "Not Significant"

df_2021 = df[df['year'] == 2021].copy()

diversity_threshold = df_2021['shannon_index'].median()
df_2021.loc[:, 'diversity_level'] = df_2021['shannon_index'].apply(
    lambda x: 'high' if x > diversity_threshold else 'low'
)

df_2021.loc[:, 'dominant_race'] = df_2021['group_classification'].str.extract(r'^(.*?) Dominant')

group_comparisons = [
    ("White", "high", "White", "low"),
    ("Black", "high", "Black", "low"),
    ("White", "high", "Black", "high"),
    ("White", "high", "Black", "low"),
    ("White", "low", "Black", "high"),
    ("White", "low", "Black", "low"),
]

results_1 = []

for race1, div1, race2, div2 in group_comparisons:
    group1 = df_2021[
        (df_2021['group_classification'].str.contains(race1, case=False, na=False)) &
        (df_2021['diversity_level'] == div1)
    ]['weighted_education_score']
    group2 = df_2021[
        (df_2021['group_classification'].str.contains(race2, case=False, na=False)) &
        (df_2021['diversity_level'] == div2)
    ]['weighted_education_score']

    if len(group1) > 0 and len(group2) > 0:
        t_stat, p_val = ttest_ind(group1, group2, nan_policy='omit')
        results_1.append({
            "Race Group 1": f"{race1.capitalize()} {div1.capitalize()}",
            "Race Group 2": f"{race2.capitalize()} {div2.capitalize()}",
            "T-Statistic": t_stat,
            "P-Value": p_val,
            "Significance": classify_p_value(p_val),
        })

results_1_df = pd.DataFrame(results_1)
results_1_df

,Race Group 1,Race Group 2,T-Statistic,P-Value,Significance
0,White High,White Low,14.173899,1.332246e-44,Highly Significant
1,Black High,Black Low,4.928664,1.152775e-06,Highly Significant
2,White High,Black High,11.256322,1.230132e-28,Highly Significant
3,White High,Black Low,13.364143,3.380989e-39,Highly Significant
4,White Low,Black High,5.568121,2.839396e-08,Highly Significant
5,White Low,Black Low,9.408543,1.101588e-20,Highly Significant


This analysis evaluates differences in weighted education scores across racial and diversity groupings in 2021. The findings reveal significant disparities and trends, with all pairwise comparisons yielding highly significant results.

### Key Findings
#### Intra-Racial Comparisons:

##### White High vs. Low Diversity:
The education score for high-diversity white-dominant communities was significantly higher than for low-diversity communities (t=14.17,p<0.0001).
This indicates that diversity within white-dominant groups strongly correlates with improved education outcomes.
##### Black High vs. Low Diversity:
Similarly, high-diversity black-dominant communities exhibited significantly higher education scores than their low-diversity counterparts (t=4.93,p<0.0001).
While still significant, the effect is less pronounced compared to white-dominant groups.
#### Inter-Racial Comparisons:

##### White High vs. Black High Diversity:
White-dominant communities with high diversity outperformed black-dominant high-diversity communities (t=11.26,p<0.0001), highlighting racial disparities even in diverse settings.
##### White High vs. Black Low Diversity:
The disparity widens when comparing white high-diversity communities to black low-diversity communities (t=13.36,p<0.0001).
#### Cross-Diversity and Cross-Race Comparisons:

##### White Low vs. Black High Diversity:
Even among low-diversity white-dominant communities and high-diversity black-dominant communities, white-dominant groups showed higher education scores (t=5.57,p<0.0001).
##### White Low vs. Black Low Diversity:
This comparison showed significant disparities (t=9.41,p<0.0001), further illustrating the advantage of white-dominant communities irrespective of diversity.

#### Implications
##### Diversity's Positive Impact:

High diversity correlates strongly with improved education scores across both racial groups, highlighting its importance in fostering equitable educational outcomes.
##### Persistent Racial Disparities:

Despite the benefits of diversity, racial disparities persist. White-dominant groups consistently achieve higher education scores than black-dominant groups, even in similar diversity contexts.

## Distribution of Education Score Categories by Diversity Levels (2021)

In [15]:
unique_education_scores_1 = df_2021['education_score_category'].unique()
unique_diversity_levels_1 = df_2021['diversity_level'].unique()

education_diversity_counts_1 = df_2021.groupby(['education_score_category', 'diversity_level']).size()

unique_education_scores, unique_diversity_levels, education_diversity_counts

(array(['Medium', 'Low', 'High', 'Very High', 'Very Low', 'nan'],
       dtype=object),
 array(['low', 'high'], dtype=object),
 education_score_category  diversity_level
 High                      high                572
                           low                 320
 Low                       high                695
                           low                 882
 Medium                    high               1075
                           low                1219
 Very High                 high                148
                           low                  66
 Very Low                  high                 20
                           low                  16
 nan                       low                  18
 dtype: int64)

The analysis examines the distribution of education score categories across high- and low-diversity groups for the year 2021. The results highlight the frequency of occurrences and differences in representation across educational categories and diversity levels.

### Key Observations
#### Education Score Categories:
The dataset includes five primary education score categories: "Very Low," "Low," "Medium," "High," and "Very High." There are also some "nan" entries indicating missing or undefined values.
Diversity levels are classified as "low" or "high" based on the Shannon Index median.
Category Distributions:

#### Medium Education:
The most common category across both diversity levels, with 1,075 cases in high-diversity areas and 1,219 cases in low-diversity areas. This indicates a balanced representation in this middle-tier category.
#### High Education:
More prevalent in high-diversity areas (572 cases) than in low-diversity areas (320 cases), suggesting a positive correlation between diversity and higher educational attainment.
#### Low Education:
Dominates low-diversity areas (882 cases) compared to high-diversity areas (695 cases), reflecting challenges in educational outcomes for low-diversity settings.
#### Very High and Very Low Education:
The "Very High" category is more frequent in high-diversity areas (148 cases) than in low-diversity areas (66 cases), underscoring diversity's positive role.
The "Very Low" category has minimal representation, with 20 cases in high-diversity areas and 16 in low-diversity ones.
#### Missing Data:

"nan" entries appear exclusively in low-diversity areas (18 cases), indicating potential data quality or classification issues in these communities.
### Implications
#### Diversity as a Catalyst:

High-diversity areas consistently exhibit higher counts in favorable education categories, such as "High" and "Very High," suggesting diversity's role in fostering educational success.
#### Challenges in Low-Diversity Areas:

Low-diversity areas show higher concentrations in the "Low" education category, indicating structural or systemic barriers that hinder educational attainment.

## Analysis of Median Income by Education Level and Gini Coefficient (2021)

In [16]:
df_2021 = df[(df['year'] == 2021) & (df['gini_education'] != -1)].dropna(subset=['median_income', 'education_score_category'])

income_education_results_1 = []

population_threshold = 30

gini_median_2021 = df_2021['gini_education'].median()

for education_level in df_2021['education_score_category'].dropna().unique():
    low_education_1 = df_2021[
        (df_2021['education_score_category'] == education_level) & 
        (df_2021['gini_education'] < gini_median_2021)
    ]['median_income']
    
    high_education_1 = df_2021[
        (df_2021['education_score_category'] == education_level) & 
        (df_2021['gini_education'] >= gini_median_2021)
    ]['median_income']

    if len(low_education_1) >= population_threshold and len(high_education_1) >= population_threshold:

        t_stat, p_val = ttest_ind(low_education_1, high_education_1, nan_policy='omit')

        combined_data_1 = pd.concat([low_education_1, high_education_1])
        z_scores = zscore(combined_data_1)

        low_education_median_1 = low_education_1.median()
        high_education_median_1 = high_education_1.median()

        income_education_results_1.append({
            "Education_Level": education_level,
            "Low_Gini_Median": low_education_median_1,
            "High_Gini_Median": high_education_median_1,
            "T-Statistic": t_stat,
            "P-Value": p_val,
            "Mean Z-Score": z_scores.mean() if len(z_scores) > 0 else None
        })

income_education_results_df_1 = pd.DataFrame(income_education_results_1)
income_education_results_df_1

,Education_Level,Low_Gini_Median,High_Gini_Median,T-Statistic,P-Value,Mean Z-Score
0,Medium,61689.0,57062.0,3.488177,0.000495,4.915125e-17
1,Low,43345.0,42584.5,0.879853,0.379079,-1.410740e-16
2,High,83941.0,87574.0,-4.074774,0.000050,1.513403e-16


This analysis evaluates the relationship between income, education levels, and Gini inequality in 2021. By comparing groups with Gini coefficients below (low inequality) and above (high inequality) the median, key trends and statistical results are observed.

### Key Findings
#### Medium Education Level:

Low Inequality: Median income for this group was 61,689.
High Inequality: Median income decreased to 57,062.
Statistical Significance: The t-statistic of 3.49 and p-value of 0.000495 indicate a highly significant difference, suggesting that lower inequality positively impacts income in medium-education communities.
#### Low Education Level:

Low Inequality: Median income was 43,345.
High Inequality: A slight decrease to 42,584.5 was observed.
Statistical Significance: The difference was not significant (t=0.88,p=0.379), indicating that inequality has minimal impact on income within the low education category.
#### High Education Level:

Low Inequality: Median income was 83,941.
High Inequality: Increased to 87,574.
Statistical Significance: The t-statistic of -4.07 and p-value of 0.00005 reveal a highly significant difference, suggesting that high inequality benefits income in high-education groups, possibly due to income stratification favoring high earners.
#### Mean Z-Score:
The near-zero mean z-scores across all categories confirm a well-centered distribution of income data, ensuring reliable statistical comparisons.
#### Implications
##### Inequality's Differential Impact:

Inequality has contrasting effects based on education levels. Low inequality benefits medium-education groups, while high inequality appears to favor high-education groups by amplifying income advantages for top earners.
##### Economic Vulnerability of Low Education Groups:

Low education groups show minimal income variation between low and high inequality, reflecting limited economic mobility and earning potential within this category.

## Pairwise Analysis of Median Income by Education Levels and Inequality (2021)

In [17]:
df_2021 = df[(df['year'] == 2021) & (df['gini_education'] != -1)].dropna(subset=['median_income', 'education_score_category'])

group_comparisons_1 = [
    ("Low", "Low", "Low", "High"),
    ("Medium", "Low", "Medium", "High"),
    ("High", "Low", "High", "High"),
    ("Low", "High", "Medium", "High"),
    ("Medium", "High", "High", "High"),
]

pairwise_results_1 = []

gini_median_2021 = df_2021['gini_education'].median()

for edu1, gini1, edu2, gini2 in group_comparisons_1:
    group1_1 = df_2021[
        (df_2021['education_score_category'] == edu1) & 
        ((df_2021['gini_education'] < gini_median_2021) if gini1 == "Low" else (df_2021['gini_education'] >= gini_median_2021))
    ]['median_income']
    
    group2_1 = df_2021[
        (df_2021['education_score_category'] == edu2) & 
        ((df_2021['gini_education'] < gini_median_2021) if gini2 == "Low" else (df_2021['gini_education'] >= gini_median_2021))
    ]['median_income']

    if len(group1_1) > 0 and len(group2_1) > 0:
        t_stat_1, p_val_1 = ttest_ind(group1_1, group2_1, nan_policy='omit')
        pairwise_results_1.append({
            "Education Level 1": f"{edu1} Education - {gini1} Inequality",
            "Education Level 2": f"{edu2} Education - {gini2} Inequality",
            "T-Statistic": t_stat_1,
            "P-Value": p_val_1,
        })

pairwise_results_df_1 = pd.DataFrame(pairwise_results_1)
pairwise_results_df_1

,Education Level 1,Education Level 2,T-Statistic,P-Value
0,Low Education - Low Inequality,Low Education - High Inequality,0.879853,3.790792e-01
1,Medium Education - Low Inequality,Medium Education - High Inequality,3.488177,4.954181e-04
2,High Education - Low Inequality,High Education - High Inequality,-4.074774,4.996014e-05
3,Low Education - High Inequality,Medium Education - High Inequality,-18.488464,1.818727e-70
4,Medium Education - High Inequality,High Education - High Inequality,-19.457477,9.708256e-75


This analysis compares median incomes across different education levels and Gini inequality classifications in 2021, highlighting statistically significant differences and trends.

### Key Findings
#### Intra-Education Comparisons:

##### Low Education, Low vs. High Inequality:
The t-statistic of 0.88 and a p-value of 0.379 indicate no significant difference in median incomes, suggesting that inequality has a minimal impact on income within the low education group.
##### Medium Education, Low vs. High Inequality:
A t-statistic of 3.49 and a p-value of 0.000495 reveal a highly significant difference, with low inequality yielding higher median incomes. This indicates that inequality significantly affects income in medium-education groups.
##### High Education, Low vs. High Inequality:
The t-statistic of -4.07 and p-value of 0.00005 demonstrate a highly significant difference, where high inequality corresponds to higher incomes, likely due to increased stratification among top earners in this group.
#### Inter-Education Comparisons:

##### Low Education (High Inequality) vs. Medium Education (High Inequality):
The t-statistic of -18.49 and p-value of 1.82×10 −70
  indicate a substantial income gap, with medium-education groups earning significantly more.
##### Medium Education (High Inequality) vs. High Education (High Inequality):
A t-statistic of -19.46 and p-value of 9.71×10 −75
  highlight the continued income advantage of higher education levels, even under high inequality conditions.
#### Implications
##### Inequality's Impact:

Inequality has a minimal effect on low education groups but significantly affects medium and high education groups, amplifying income disparities in these categories.
##### Educational Advancement:

Higher education consistently correlates with increased incomes, even under conditions of high inequality. This reinforces the economic value of educational attainment.

## Pairwise Analysis of Median Income by Education Levels and Inequality (2021)

In [18]:
def classify_p_value_1(p_val):
    if p_val <= 0.01:
        return "Highly Significant"
    elif 0.01 < p_val <= 0.05:
        return "Significant"
    elif 0.05 < p_val <= 0.1:
        return "Marginally Significant"
    else:
        return "Not Significant"

df_2021 = df[(df['year'] == 2021) & (df['gini_education'] != -1)].dropna(subset=['median_income', 'education_score_category'])

education_levels_1 = df_2021['education_score_category'].unique()
gini_categories_1 = ["low", "high"]

group_comparisons_1 = list(product(education_levels_1, gini_categories_1, education_levels_1, gini_categories_1))

education_results_1 = []

gini_median_1 = df_2021['gini_education'].median()

for edu1_1, gini1_1, edu2_1, gini2_1 in group_comparisons_1:

    if edu1_1 == edu2_1 and gini1_1 == gini2_1:
        continue

    group1_1 = df_2021[
        (df_2021['education_score_category'] == edu1_1) &
        ((df_2021['gini_education'] < gini_median_1) if gini1_1 == "low" else (df_2021['gini_education'] >= gini_median_1))
    ]['median_income']
    
    group2_1 = df_2021[
        (df_2021['education_score_category'] == edu2_1) &
        ((df_2021['gini_education'] < gini_median_1) if gini2_1 == "low" else (df_2021['gini_education'] >= gini_median_1))
    ]['median_income']

    if len(group1_1) > 0 and len(group2_1) > 0:
        t_stat_1, p_val_1 = ttest_ind(group1_1, group2_1, nan_policy='omit')
        education_results_1.append({
            "Education Level 1": f"{edu1_1} Education Level ({'Low' if gini1_1 == 'low' else 'High'})",
            "Education Level 2": f"{edu2_1} Education Level ({'Low' if gini2_1 == 'low' else 'High'})",
            "T-Statistic": t_stat_1,
            "P-Value": p_val_1,
            "Significance": classify_p_value_1(p_val_1),
        })

cleaned_results_1 = []
seen_combinations_1 = set()

for result_1 in education_results_1:
    combo_1 = tuple(sorted([result_1["Education Level 1"], result_1["Education Level 2"]]))
    if combo_1 not in seen_combinations_1:
        seen_combinations_1.add(combo_1)
        cleaned_results_1.append(result_1)

unique_education_results_df_1 = pd.DataFrame(cleaned_results_1)
unique_education_results_df_1

,Education Level 1,Education Level 2,T-Statistic,P-Value,Significance
0,Medium Education Level (Low),Medium Education Level (High),3.488177,4.954181e-04,Highly Significant
1,Medium Education Level (Low),Low Education Level (Low),17.272491,3.999017e-62,Highly Significant
2,Medium Education Level (Low),Low Education Level (High),23.703529,7.767549e-112,Highly Significant
3,Medium Education Level (Low),High Education Level (Low),-16.766942,7.066676e-59,Highly Significant
4,Medium Education Level (Low),High Education Level (High),-19.909853,4.816989e-80,Highly Significant
5,Medium Education Level (Low),Very High Education Level (High),-28.988530,6.813957e-149,Highly Significant
6,Medium Education Level (Low),Very Low Education Level (Low),3.167335,1.571887e-03,Highly Significant
7,Medium Education Level (Low),Very Low Education Level (High),5.711627,1.363600e-08,Highly Significant
8,Medium Education Level (High),Low Education Level (Low),13.870048,4.271997e-41,Highly Significant
9,Medium Education Level (High),Low Education Level (High),18.488464,1.818727e-70,Highly Significant


This analysis compares median incomes across various education levels and inequality conditions. Key findings reveal significant differences that shed light on the impact of education and inequality on economic outcomes.

### Key Findings
#### Intra-Education Comparisons:

##### Medium Education Level:
Low vs. High Inequality: A t-statistic of 3.49 (p=0.0005) indicates a highly significant difference, with lower inequality favoring higher incomes.
##### Low Education Level:
Low vs. High Inequality: Results were not significant (t=0.88,p=0.379), suggesting minimal impact of inequality within this group.
##### High Education Level:
Low vs. High Inequality: A t-statistic of -4.07 (p=0.00005) shows a highly significant difference, where higher inequality corresponds to higher median incomes, reflecting increased income stratification among top earners.
#### Inter-Education Comparisons:

##### Low vs. Medium Education:
Medium education consistently outperformed low education in income across all inequality conditions. For example, comparing "Medium (Low Inequality)" to "Low (High Inequality)" yielded a t-statistic of 23.70 (p<0.0001).
##### Medium vs. High Education:
High education groups outperformed medium groups, particularly under high inequality. The comparison between "Medium (High Inequality)" and "High (High Inequality)" yielded a t-statistic of -19.46 (p<0.0001).
##### High vs. Very High Education:
The gap between high and very high education groups is narrower but still significant, with a t-statistic of -9.55 (p<0.0001).
#### Edge Cases:

Comparisons involving "Very Low Education" groups often yielded less significant differences. For example, the comparison between "Very Low (Low Inequality)" and "Very Low (High Inequality)" was not significant (t=−0.01,p=0.989).
#### Implications
##### Inequality's Impact:

Inequality has minimal impact on income for low education groups but significantly influences medium and high education groups, exacerbating income disparities.
##### Educational Mobility:

Advancing education levels consistently correlates with higher incomes, even in high-inequality scenarios, underscoring the economic value of education.

## Income Distribution Analysis by Education Score Categories (2021)

In [19]:
education_category_analysis_2021 = df_2021.groupby('education_score_category').agg(
    Median_Income=('median_income', 'median'),
    Std_Income=('median_income', 'std')
).reset_index()

education_category_analysis_2021

,education_score_category,Median_Income,Std_Income
0,High,85556.0,37553.025615
1,Low,42713.0,16351.968954
2,Medium,60302.0,22035.992762
3,Very High,128875.0,84002.482300
4,Very Low,37014.0,16981.814622


This analysis examines the median and standard deviation of incomes across different education score categories in 2021, providing insights into income levels and variability within each group.

### Key Observations
#### Median Income:

##### Very High Education: 
The highest median income was 128,875, indicating substantial economic benefits associated with top-tier education.
##### High Education: 
The second-highest median income of 85,556 highlights the strong financial advantages of achieving a high education level.
##### Medium Education:
The median income was 60,302, reflecting a moderate economic position for this category.
##### Low and Very Low Education:
These categories had the lowest median incomes, at 42,713 and 37,014 respectively, underscoring the financial challenges faced by individuals with lower educational attainment.
#### Income Variability (Standard Deviation):

##### Very High Education:
The largest standard deviation (84,002.48) suggests significant income variability, potentially influenced by high earners within this group.
##### High and Medium Education:
These categories exhibited moderate variability, with standard deviations of 37,553.03 and 22,035.99 respectively, reflecting relatively consistent income distributions.
##### Low and Very Low Education:
The smallest standard deviations (16,351.97 and 16,981.81) suggest limited income diversity, consistent with constrained earning potential in these groups.
#### Implications
##### Economic Returns on Education:

Median incomes increase substantially with higher education levels, reinforcing the economic value of investing in education.
##### Inequality in High Education Groups:

The high standard deviation in the "Very High" education category highlights potential income inequality even among highly educated individuals.

## ANOVA Analysis of Median Income by Diversity Levels (2021)

In [20]:
df_2021_cleaned_1 = df[(df['year'] == 2021) & (df['higher_education_proportion'] >= 0)].dropna(
    subset=['higher_education_proportion', 'group_classification', 'median_income']
)

group_classifications_1 = df_2021_cleaned_1['group_classification'].unique()

anova_results_with_significance_1 = []

def classify_p_value_1(p_val):
    if p_val <= 0.01:
        return "Highly Significant"
    elif 0.01 < p_val <= 0.05:
        return "Significant"
    elif 0.05 < p_val <= 0.1:
        return "Marginally Significant"
    else:
        return "Not Significant"

for group1_1 in group_classifications_1:
    for group2_1 in group_classifications_1:
        if group1_1 != group2_1:
            group1_data_1 = df_2021_cleaned_1[df_2021_cleaned_1['group_classification'] == group1_1]['median_income']
            group2_data_1 = df_2021_cleaned_1[df_2021_cleaned_1['group_classification'] == group2_1]['median_income']

            if len(group1_data_1) > 30 and len(group2_data_1) > 30:
           
                t_stat_1, p_val_1 = ttest_ind(group1_data_1, group2_data_1, nan_policy='omit')

                combined_data_1 = pd.concat([group1_data_1, group2_data_1])
                z_scores_1 = zscore(combined_data_1)

                anova_results_with_significance_1.append({
                    "Diversity Level": group1_1,
                    "Diversity Level": group2_1,
                    "T-Statistic": t_stat_1,
                    "P-Value": p_val_1,
                    "Mean Z-Score": z_scores_1.mean() if len(z_scores_1) > 0 else None,
                    "Significance": classify_p_value_1(p_val_1)
                })

anova_results_with_significance_df_1 = pd.DataFrame(anova_results_with_significance_1)
anova_results_with_significance_df_1

,Diversity Level,T-Statistic,P-Value,Mean Z-Score,Significance
0,white Dominant with high diversity,-5.343664,9.581529e-08,-1.325330e-17,Highly Significant
1,Mixed Race Dominant,6.365240,2.226736e-10,2.650455e-17,Highly Significant
2,black Dominant with low diversity,13.640188,2.816934e-41,-6.629768e-17,Highly Significant
3,black Dominant with high diversity,4.987893,6.442829e-07,5.078603e-17,Highly Significant
4,white Dominant with low diversity,5.343664,9.581529e-08,-2.981993e-17,Highly Significant
5,Mixed Race Dominant,7.999353,2.460153e-15,-1.028417e-16,Highly Significant
6,black Dominant with low diversity,15.097872,2.178491e-48,-8.040686e-17,Highly Significant
7,black Dominant with high diversity,5.920216,4.052312e-09,-1.131929e-16,Highly Significant
8,white Dominant with low diversity,-6.365240,2.226736e-10,2.650455e-17,Highly Significant
9,white Dominant with high diversity,-7.999353,2.460153e-15,-1.121910e-16,Highly Significant


This analysis explores median income disparities across different diversity classifications using pairwise comparisons and ANOVA testing. Significant differences are highlighted to identify key trends.

### Key Findings
#### Highly Significant Comparisons:

##### White-Dominant vs. Black-Dominant:
Across both high and low diversity levels, white-dominant groups consistently exhibit higher median incomes compared to black-dominant groups. For example:
"White Dominant with High Diversity" vs. "Black Dominant with Low Diversity" (t = −15.10, p <0.0001).
##### Mixed Race Dominant vs. Other Groups:
Mixed-race dominant groups also show significantly higher incomes compared to black-dominant groups, regardless of diversity levels (t = 7.99, p < 0.0001).
##### Black High vs. Low Diversity:
Black-dominant groups with high diversity exhibit higher incomes than those with low diversity (t = 5.92, p < 0.0001).
#### Significant Comparisons:

##### Black-Dominant with High Diversity vs. Low Diversity:
A t-statistic of 2.55 (p = 0.011) indicates a significant but less pronounced improvement in income for high diversity groups compared to low diversity ones.
#### Not Significant Comparisons:

##### Black-Dominant vs. Mixed-Race Dominant:
In some cases, differences were not significant, such as "Mixed Race Dominant" vs. "Black Dominant with High Diversity" (t = −1.43, p = 0.155).
#### Z-Scores:

Near-zero mean z-scores across all comparisons confirm that income distributions are appropriately normalized and centered.
#### Implications
##### Diversity as a Positive Factor:

High diversity correlates positively with higher median incomes, especially within black-dominant groups. Policies promoting diversity could improve economic outcomes for these communities.
##### Racial Disparities:

Significant income disparities persist between white- and black-dominant groups, highlighting systemic inequities that need addressing.

# Year 2022 Analysis

In [21]:
df_2022 = df[(df['year'] == 2022) & (df['gini_coefficient'] != -1)].dropna(subset=['median_income', 'group_classification'])

df_2022['dominant_race'] = df_2022['group_classification'].str.extract(r'^(.*?) Dominant')
df_2022['diversity_level'] = df_2022['group_classification'].str.extract(r'with (low|high) diversity')

income_race_results_2 = []

population_threshold = 30

for race in df_2022['dominant_race'].dropna().unique():
    low_diversity = df_2022[
        (df_2022['dominant_race'] == race) & 
        (df_2022['diversity_level'] == 'low')
    ]['median_income']
    
    high_diversity = df_2022[
        (df_2022['dominant_race'] == race) & 
        (df_2022['diversity_level'] == 'high')
    ]['median_income']
    
    if len(low_diversity) >= population_threshold and len(high_diversity) >= population_threshold:
        t_stat, p_val = ttest_ind(low_diversity, high_diversity, nan_policy='omit')

        combined_data = pd.concat([low_diversity, high_diversity])
        z_scores = zscore(combined_data)

        low_diversity_median = low_diversity.median()
        high_diversity_median = high_diversity.median()

        income_race_results_2.append({
            "Dominant_Race": race,
            "Low_Diversity_Median": low_diversity_median,
            "High_Diversity_Median": high_diversity_median,
            "T-Statistic": t_stat,
            "P-Value": p_val,
            "Mean Z-Score": z_scores.mean() if len(z_scores) > 0 else None
        })

income_race_results_2_df = pd.DataFrame(income_race_results_2)
income_race_results_2_df

,Dominant_Race,Low_Diversity_Median,High_Diversity_Median,T-Statistic,P-Value,Mean Z-Score
0,white,63235.0,70597.5,-5.803414,6.967565e-09,-1.460588e-16
1,black,41294.0,46337.0,-2.779244,5.669300e-03,-7.607524e-17


In [22]:
def classify_p_value(p_val):
    if p_val <= 0.01:
        return "Highly Significant"
    elif 0.01 < p_val <= 0.05:
        return "Significant"
    elif 0.05 < p_val <= 0.1:
        return "Marginally Significant"
    else:
        return "Not Significant"

df_2022 = df[df['year'] == 2022].copy()

diversity_threshold = df_2022['shannon_index'].median()
df_2022.loc[:, 'diversity_level'] = df_2022['shannon_index'].apply(
    lambda x: 'high' if x > diversity_threshold else 'low'
)

df_2022.loc[:, 'dominant_race'] = df_2022['group_classification'].str.extract(r'^(.*?) Dominant')

group_comparisons = [
    ("White", "high", "White", "low"),
    ("Black", "high", "Black", "low"),
    ("White", "high", "Black", "high"),
    ("White", "high", "Black", "low"),
    ("White", "low", "Black", "high"),
    ("White", "low", "Black", "low"),
]

results_2 = []

for race1, div1, race2, div2 in group_comparisons:
    group1 = df_2022[
        (df_2022['group_classification'].str.contains(race1, case=False, na=False)) &
        (df_2022['diversity_level'] == div1)
    ]['weighted_education_score']
    group2 = df_2022[
        (df_2022['group_classification'].str.contains(race2, case=False, na=False)) &
        (df_2022['diversity_level'] == div2)
    ]['weighted_education_score']

    if len(group1) > 0 and len(group2) > 0:
        t_stat, p_val = ttest_ind(group1, group2, nan_policy='omit')
        results_2.append({
            "Race Group 1": f"{race1.capitalize()} {div1.capitalize()}",
            "Race Group 2": f"{race2.capitalize()} {div2.capitalize()}",
            "T-Statistic": t_stat,
            "P-Value": p_val,
            "Significance": classify_p_value(p_val),
        })

results_2_df = pd.DataFrame(results_2)
results_2_df

,Race Group 1,Race Group 2,T-Statistic,P-Value,Significance
0,White High,White Low,13.319329,1.090336e-39,Highly Significant
1,Black High,Black Low,5.467481,7.462475e-08,Highly Significant
2,White High,Black High,10.059228,2.578172e-23,Highly Significant
3,White High,Black Low,13.002103,2.889008e-37,Highly Significant
4,White Low,Black High,4.776261,1.885337e-06,Highly Significant
5,White Low,Black Low,9.559060,2.737829e-21,Highly Significant


In [23]:
unique_education_scores_2 = df_2022['education_score_category'].unique()
unique_diversity_levels_2 = df_2022['diversity_level'].unique()

education_diversity_counts_2 = df_2022.groupby(['education_score_category', 'diversity_level']).size()

unique_education_scores, unique_diversity_levels, education_diversity_counts

(array(['Medium', 'Low', 'High', 'Very High', 'Very Low', 'nan'],
       dtype=object),
 array(['low', 'high'], dtype=object),
 education_score_category  diversity_level
 High                      high                572
                           low                 320
 Low                       high                695
                           low                 882
 Medium                    high               1075
                           low                1219
 Very High                 high                148
                           low                  66
 Very Low                  high                 20
                           low                  16
 nan                       low                  18
 dtype: int64)

In [24]:
df_2022 = df[(df['year'] == 2022) & (df['gini_education'] != -1)].dropna(subset=['median_income', 'education_score_category'])

income_education_results_2 = []

population_threshold_2 = 30

gini_median_2022 = df_2022['gini_education'].median()

for education_level_2 in df_2022['education_score_category'].dropna().unique():
    low_education_2 = df_2022[
        (df_2022['education_score_category'] == education_level_2) & 
        (df_2022['gini_education'] < gini_median_2022)
    ]['median_income']
    
    high_education_2 = df_2022[
        (df_2022['education_score_category'] == education_level_2) & 
        (df_2022['gini_education'] >= gini_median_2022)
    ]['median_income']

    if len(low_education_2) >= population_threshold_2 and len(high_education_2) >= population_threshold_2:

        t_stat_2, p_val_2 = ttest_ind(low_education_2, high_education_2, nan_policy='omit')

        combined_data_2 = pd.concat([low_education_2, high_education_2])
        z_scores_2 = zscore(combined_data_2)

        low_education_median_2 = low_education_2.median()
        high_education_median_2 = high_education_2.median()

        income_education_results_2.append({
            "Education_Level": education_level_2,
            "Low_Gini_Median": low_education_median_2,
            "High_Gini_Median": high_education_median_2,
            "T-Statistic": t_stat_2,
            "P-Value": p_val_2,
            "Mean Z-Score": z_scores_2.mean() if len(z_scores_2) > 0 else None
        })

income_education_results_df_2 = pd.DataFrame(income_education_results_2)
income_education_results_df_2

,Education_Level,Low_Gini_Median,High_Gini_Median,T-Statistic,P-Value,Mean Z-Score
0,Low,47344.0,45819.0,1.326961,0.184734,1.664558e-16
1,Medium,65742.0,60938.0,4.201579,0.000027,8.331322e-17
2,High,91188.0,91898.5,-2.736264,0.006333,-1.144806e-16


In [25]:
df_2022 = df[(df['year'] == 2022) & (df['gini_education'] != -1)].dropna(subset=['median_income', 'education_score_category'])

group_comparisons_2 = [
    ("Low", "Low", "Low", "High"),
    ("Medium", "Low", "Medium", "High"),
    ("High", "Low", "High", "High"),
    ("Low", "High", "Medium", "High"),
    ("Medium", "High", "High", "High"),
]

pairwise_results_2 = []

gini_median_2 = df_2022['gini_education'].median()

for edu1_2, gini1_2, edu2_2, gini2_2 in group_comparisons_2:
    group1_2 = df_2022[
        (df_2022['education_score_category'] == edu1_2) & 
        ((df_2022['gini_education'] < gini_median_2) if gini1_2 == "Low" else (df_2022['gini_education'] >= gini_median_2))
    ]['median_income']
    
    group2_2 = df_2022[
        (df_2022['education_score_category'] == edu2_2) & 
        ((df_2022['gini_education'] < gini_median_2) if gini2_2 == "Low" else (df_2022['gini_education'] >= gini_median_2))
    ]['median_income']

    if len(group1_2) > 0 and len(group2_2) > 0:
        t_stat_2, p_val_2 = ttest_ind(group1_2, group2_2, nan_policy='omit')
        pairwise_results_2.append({
            "Education Level 1": f"{edu1} Education - {gini1} Inequality",
            "Education Level 2": f"{edu2} Education - {gini2} Inequality",
            "T-Statistic": t_stat_2,
            "P-Value": p_val_2,
        })

pairwise_results_df_2 = pd.DataFrame(pairwise_results_2)
pairwise_results_df_2

,Education Level 1,Education Level 2,T-Statistic,P-Value
0,Medium Education - High Inequality,High Education - High Inequality,1.326961,1.847336e-01
1,Medium Education - High Inequality,High Education - High Inequality,4.201579,2.748052e-05
2,Medium Education - High Inequality,High Education - High Inequality,-2.736264,6.332649e-03
3,Medium Education - High Inequality,High Education - High Inequality,-16.621593,3.445873e-58
4,Medium Education - High Inequality,High Education - High Inequality,-18.929826,1.623736e-71


In [26]:
def classify_p_value_2(p_val):
    if p_val <= 0.01:
        return "Highly Significant"
    elif 0.01 < p_val <= 0.05:
        return "Significant"
    elif 0.05 < p_val <= 0.1:
        return "Marginally Significant"
    else:
        return "Not Significant"

df_2022 = df[(df['year'] == 2022) & (df['gini_education'] != -1)].dropna(subset=['median_income', 'education_score_category'])

education_levels_2 = df_2022['education_score_category'].unique()
gini_categories_2 = ["low", "high"]

group_comparisons_2 = list(product(education_levels_2, gini_categories_2, education_levels_2, gini_categories_2))

education_results_2 = []

gini_median_2 = df_2022['gini_education'].median()

for edu1_2, gini1_2, edu2_2, gini2_2 in group_comparisons_2:

    if edu1_2 == edu2_2 and gini1_2 == gini2_2:
        continue

    group1_2 = df_2022[
        (df_2022['education_score_category'] == edu1_2) &
        ((df_2022['gini_education'] < gini_median_2) if gini1_2 == "low" else (df_2022['gini_education'] >= gini_median_2))
    ]['median_income']
    
    group2_2 = df_2022[
        (df_2022['education_score_category'] == edu2_2) &
        ((df_2022['gini_education'] < gini_median_2) if gini2_2 == "low" else (df_2022['gini_education'] >= gini_median_2))
    ]['median_income']

    if len(group1_2) > 0 and len(group2_2) > 0:
        t_stat_2, p_val_2 = ttest_ind(group1_2, group2_2, nan_policy='omit')
        education_results_2.append({
            "Education Level 1": f"{edu1_2} Education Level ({'Low' if gini1_2 == 'low' else 'High'})",
            "Education Level 2": f"{edu2_2} Education Level ({'Low' if gini2_2 == 'low' else 'High'})",
            "T-Statistic": t_stat_2,
            "P-Value": p_val_2,
            "Significance": classify_p_value_2(p_val_2),
        })

cleaned_results_2 = []
seen_combinations_2 = set()

for result_2 in education_results_2:
    combo_2 = tuple(sorted([result_2["Education Level 1"], result_2["Education Level 2"]]))
    if combo_2 not in seen_combinations_2:
        seen_combinations_2.add(combo_2)
        cleaned_results_2.append(result_2)

unique_education_results_df_2 = pd.DataFrame(cleaned_results_2)
unique_education_results_df_2

,Education Level 1,Education Level 2,T-Statistic,P-Value,Significance
0,Low Education Level (Low),Low Education Level (High),1.326961,1.847336e-01,Not Significant
1,Low Education Level (Low),Medium Education Level (Low),-15.709858,2.893973e-52,Highly Significant
2,Low Education Level (Low),Medium Education Level (High),-11.682690,3.586719e-30,Highly Significant
3,Low Education Level (Low),High Education Level (Low),-27.108544,3.606401e-119,Highly Significant
4,Low Education Level (Low),High Education Level (High),-20.501096,1.293830e-76,Highly Significant
5,Low Education Level (Low),Very High Education Level (High),-22.471856,1.231121e-83,Highly Significant
6,Low Education Level (Low),Very Low Education Level (Low),2.052469,4.071781e-02,Significant
7,Low Education Level (Low),Very Low Education Level (High),4.527246,7.677303e-06,Highly Significant
8,Low Education Level (High),Medium Education Level (Low),-22.612708,8.848793e-103,Highly Significant
9,Low Education Level (High),Medium Education Level (High),-16.621593,3.445873e-58,Highly Significant


In [27]:
education_category_analysis_2022 = df_2022.groupby('education_score_category').agg(
    Median_Income=('median_income', 'median'),
    Std_Income=('median_income', 'std')
).reset_index()

education_category_analysis_2022

,education_score_category,Median_Income,Std_Income
0,High,91316.0,41473.881936
1,Low,46107.5,18154.519882
2,Medium,64045.5,23841.849755
3,Very High,131444.5,90621.611655
4,Very Low,30123.5,15868.384112


In [28]:
df_2022_cleaned_2 = df[(df['year'] == 2022) & (df['higher_education_proportion'] >= 0)].dropna(
    subset=['higher_education_proportion', 'group_classification', 'median_income']
)

group_classifications_2 = df_2022_cleaned_2['group_classification'].unique()

anova_results_with_significance_2 = []

def classify_p_value_2(p_val):
    if p_val <= 0.01:
        return "Highly Significant"
    elif 0.01 < p_val <= 0.05:
        return "Significant"
    elif 0.05 < p_val <= 0.1:
        return "Marginally Significant"
    else:
        return "Not Significant"

for group1_2 in group_classifications_2:
    for group2_2 in group_classifications_2:
        if group1_2 != group2_2:
            group1_data_2 = df_2022_cleaned_2[df_2022_cleaned_2['group_classification'] == group1_2]['median_income']
            group2_data_2 = df_2022_cleaned_2[df_2022_cleaned_2['group_classification'] == group2_2]['median_income']

            if len(group1_data_2) > 30 and len(group2_data_2) > 30:

                t_stat_2, p_val_2 = ttest_ind(group1_data_2, group2_data_2, nan_policy='omit')

                combined_data_2 = pd.concat([group1_data_2, group2_data_2])
                z_scores_2 = zscore(combined_data_2)

                anova_results_with_significance_2.append({
                    "Diversity Level": group1_2,
                    "Diversity Level": group2_2,
                    "T-Statistic": t_stat_2,
                    "P-Value": p_val_2,
                    "Mean Z-Score": z_scores_2.mean() if len(z_scores_2) > 0 else None,
                    "Significance": classify_p_value_2(p_val_2)
                })

anova_results_with_significance_df_2 = pd.DataFrame(anova_results_with_significance_2)
anova_results_with_significance_df_2

,Diversity Level,T-Statistic,P-Value,Mean Z-Score,Significance
0,white Dominant with low diversity,-6.045568,1.669298e-09,5.582005e-17,Highly Significant
1,white Dominant with high diversity,-8.564260,2.425633e-17,-3.391612e-17,Highly Significant
2,black Dominant with low diversity,5.346748,1.275339e-07,-4.729070e-17,Highly Significant
3,black Dominant with high diversity,0.960012,3.377925e-01,7.354344e-17,Not Significant
4,Mixed Race Dominant,6.045568,1.669298e-09,5.116838e-17,Highly Significant
5,white Dominant with high diversity,-5.721357,1.128887e-08,1.327187e-16,Highly Significant
6,black Dominant with low diversity,12.586500,1.707022e-35,-7.641435e-17,Highly Significant
7,black Dominant with high diversity,4.522930,6.342171e-06,1.508170e-16,Highly Significant
8,Mixed Race Dominant,8.564260,2.425633e-17,-1.695806e-17,Highly Significant
9,white Dominant with low diversity,5.721357,1.128887e-08,1.526265e-16,Highly Significant


## Notable Trends in Significance

From the heatmaps generated for 2020, 2021, and 2022, we can observe some notable trends in statistical significance across the years:

#### 1. Highly Significant Comparisons:
2020: Some pairs of education levels (e.g., "Low" vs "High") show highly significant differences, as indicated by low P-Values.

2021 and 2022: These trends persist, with similar education pairs maintaining high significance, but there might be shifts in the strength of significance for other pairs.

#### 2. Emergence of Marginal Significance:
2021 and 2022: Comparisons that were highly significant in 2020 may have shifted to marginal significance in subsequent years. This suggests potential changes in the underlying data distribution or variability.

#### 3. Significance for "Low" vs "Medium" and "Medium" vs "High":
2021: Comparisons between adjacent education categories (e.g., "Low" vs "Medium") consistently show significance, reflecting meaningful but incremental differences in median income.

2022: These comparisons remain significant, highlighting stability in these relationships over time.

#### 4. Decreasing Significance for Distant Categories:
2022: Comparisons involving "Very Low" or "Very High" education levels might show reduced significance compared to earlier years, potentially indicating greater variance or less pronounced differences.

#### 5. Clustering of Similar P-Values:
Across all years, certain group comparisons (e.g., "Low-High" across Gini categories) consistently yield similar P-Values, suggesting a stable relationship irrespective of the year.

These trends could indicate broader socio-economic patterns, such as shifting income distributions or changing impacts of educational attainment over time.

In [29]:
median_income_summary = {
    "Education Levels": [],
    "2020 Median Income": [],
    "2021 Median Income": [],
    "2022 Median Income": [],
}

for df_year, year_label in zip(
    [df[(df['year'] == 2020)], df[(df['year'] == 2021)], df[(df['year'] == 2022)]],
    ["2020", "2021", "2022"]
):
    for education_level in df_year['education_score_category'].unique():
        if education_level not in median_income_summary["Education Levels"]:
            median_income_summary["Education Levels"].append(education_level)
            median_income_summary["2020 Median Income"].append(None)
            median_income_summary["2021 Median Income"].append(None)
            median_income_summary["2022 Median Income"].append(None)
        idx = median_income_summary["Education Levels"].index(education_level)
        median_income_summary[f"{year_label} Median Income"][idx] = df_year[
            df_year['education_score_category'] == education_level
        ]['median_income'].median()

median_income_df = pd.DataFrame(median_income_summary)
median_income_df

,Education Levels,2020 Median Income,2021 Median Income,2022 Median Income
0,Medium,57578.0,60302.0,64045.5
1,Low,40368.0,42713.0,46107.5
2,High,82297.0,85556.0,91316.0
3,Very High,117779.5,128875.0,131444.5
4,Very Low,33137.5,37014.0,30123.5
5,nan,0.0,0.0,0.0


In [30]:
income_change_summary_rewritten = {
    "Education Levels": [],
    "2020 to 2021 Change (%)": [],
    "2021 to 2022 Change (%)": [],
}

for _, row in median_income_df.iterrows():
    education_level = row["Education Levels"]
    income_2020 = row["2020 Median Income"]
    income_2021 = row["2021 Median Income"]
    income_2022 = row["2022 Median Income"]
 
    income_change_summary_rewritten["Education Levels"].append(education_level)
 
    if pd.notna(income_2020) and pd.notna(income_2021) and income_2020 != 0:
        change_2020_2021 = ((income_2021 - income_2020) / income_2020) * 100
    else:
        change_2020_2021 = None 
    income_change_summary_rewritten["2020 to 2021 Change (%)"].append(change_2020_2021)
   
    if pd.notna(income_2021) and pd.notna(income_2022) and income_2021 != 0:
        change_2021_2022 = ((income_2022 - income_2021) / income_2021) * 100
    else:
        change_2021_2022 = None
    income_change_summary_rewritten["2021 to 2022 Change (%)"].append(change_2021_2022)

income_change_df_rewritten = pd.DataFrame(income_change_summary_rewritten)

final_filtered_income_change_df = income_change_df_rewritten.dropna(subset=["Education Levels"])
final_filtered_income_change_df = final_filtered_income_change_df[
    final_filtered_income_change_df["Education Levels"] != "nan"
]

final_filtered_income_change_df

,Education Levels,2020 to 2021 Change (%),2021 to 2022 Change (%)
0,Medium,4.730974,6.207920
1,Low,5.809057,7.947229
2,High,3.960047,6.732433
3,Very High,9.420570,1.993792
4,Very Low,11.698227,-18.615929
